# Replace exchange-correlation by PBE and rVV10

In [1]:
import _qepy
import qepy
import importlib

In [2]:
import numpy as np
from qepy.driver import Driver
from qepy.io import QEInput

In [3]:
from ase.io.trajectory import Trajectory
from ase.lattice.hexagonal import Graphene
from ase import Atoms
import matplotlib.pyplot as plt

In [4]:
qe_options = {
    '&control': {
        'calculation': "'scf'",
        'pseudo_dir': "'./'"
    },
    '&system': {
        'ibrav' : 0,
        'degauss': 0.005,
        'ecutwfc': 30,
        'nat': 1,
        'ntyp': 1,
        'occupations': "'smearing'",
        'input_dft' : "'PBE'"
    },
    '&electrons': {
        'conv_thr' : 1e-8
    },
    'atomic_positions crystal': ['Al    0.0  0.0  0.0'],
    'atomic_species': ['Al  26.98 Al_ONCV_PBE-1.2.upf'],
    'k_points automatic': ['4 4 4 0 0 0'],
    'cell_parameters angstrom':[
        '0.     2.025  2.025',
        '2.025  0.     2.025',
        '2.025  2.025  0.   '],
}

In [5]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.out')

In [6]:
for i in range(60):
    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('Iter: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break
driver.get_energy()

Iter:  0  - Conv:  0.07423450252710213
Iter:  1  - Conv:  0.0014686764583955348
Iter:  2  - Conv:  3.4654773793272354e-05
Iter:  3  - Conv:  6.680306265302417e-08
Iter:  4  - Conv:  2.5412863642447476e-09


-137.91449178319868

In [7]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp2.out')

In [8]:
from dftpy.functional import XC
def eval_xc(driver):
    rho = driver.get_density()
    field = driver.data2field(rho)
    func = xc(field)
    pot = driver.field2data(func.potential)*2
    energy = func.energy*2
    return pot, energy

In [9]:
xc = XC('PBE')
for i in range(60):
    extpot, ex = eval_xc(driver)
    driver.set_external_potential(potential=extpot, extene=ex, exttype=4)
    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('Iter: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break

driver.get_energy()

Iter:  0  - Conv:  0.07424296727173395
Iter:  1  - Conv:  0.001468217196695493
Iter:  2  - Conv:  3.469166657005103e-05
Iter:  3  - Conv:  7.047338781552066e-08
Iter:  4  - Conv:  2.171360641664497e-09


-137.91479262069387

In [10]:
xc = XC('RVV10')
for i in range(60):
    extpot, ex = eval_xc(driver)
    driver.set_external_potential(potential=extpot, extene=ex, exttype=4)
    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('Iter: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break

driver.get_energy()

Iter:  0  - Conv:  1.822818971869831e-05
Iter:  1  - Conv:  1.0555203104114239e-06
Iter:  2  - Conv:  1.6239999455114602e-08
Iter:  3  - Conv:  1.384650987505714e-10


-138.0117587181245